In [1]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel, AutoConfig

# Model ID
model_id = "meta-llama/Llama-3.2-1B"


# Load tokenizer and model with the updated config
tok = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)

# Input text and tokenization
text = "The dog chased another dog"
tokens = tok(text, return_tensors="pt")["input_ids"]

# Extract embeddings
embeddings = model.embed_tokens(tokens)
hdim = embeddings.shape[-1]

# Define projection layers and multihead attention
W_q = nn.Linear(hdim, hdim, bias=False)
W_k = nn.Linear(hdim, hdim, bias=False)
W_v = nn.Linear(hdim, hdim, bias=False)
mha = nn.MultiheadAttention(embed_dim=hdim, num_heads=4, batch_first=True)

# Initialize weights
with torch.no_grad():
    for param in mha.parameters():
        nn.init.normal_(param, std=0.1)

# Compute attention outputs
output, _ = mha(W_q(embeddings), W_k(embeddings), W_v(embeddings))

# Compare outputs for "dog"
dog1_out = output[0, 2]
dog2_out = output[0, 5]
print(f"Dog output identical?: {torch.allclose(dog1_out, dog2_out, atol=1e-6)}")  # True


/Users/rshankar/anaconda3/envs/cv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/rshankar/anaconda3/envs/cv/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <5AA8DD3D-A2CC-31CA-8060-88B4E9C18B09> /Users/rshankar/anaconda3/envs/cv/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <EEB3232B-F6A7-3262-948C-BB2F54905803> /Users/rshankar/anaconda3/envs/cv/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/rshankar/anaconda3/envs/cv/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvi

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Dog output identical?: True
